# Introduction

In [7]:
library(tidyverse)
library(tidymodels)
library(lubridate)
library(dbplyr)
library(ggplot2)

players<- read_csv("players.csv")
head(players)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


### Summary of Variable names
|Number of Variable | Variable Name| Type of Variable | Meaning of The Variable |
| ------------------| ------------ | ---------------- | ----------------------- |
|1                  | hashedEmail  | Character (chr)  | A unique identifier for each player|
|2                  | experience   | Character (chr)  | Player's level of experience |
|3                  | subscribe    | Logical (lgl)	  | TRUE/FALSE indicating whether the player is subscribed to a game-related newsletter	|
|4                  | played_hours | Numeric (dbl)    | Total number of hours the player has spent playing the game for the current session |
|5                  | name         | Character (chr)  | The name of the player|
|6                 | gender       | Character (chr)  | The gender of the player	|
|7                 | Age          | Numeric (dbl)    | The age of the player	|


### Issues
| Issue | Description of Issue | Solution |
| ------| -------------------- | -------- |
| Duplicate Entries | Some participants have multiple records | Ensure these are distinct play sessions and not duplicates. Group data by hashedEmail to verify if multiple sessions correspond to the same user or if it's a data entry error	|
| Missing Data | Missing values in key columns like start_time, end_time, experience, subscribe, played_hours | Ignore missing values drop_na(), Fill in missing values using mean imputation step_impute_mean(all_predictors()) |
| Time Format Consistency | The start_time and end_time columns use a string format | Convert these strings to a proper datetime format |
| Age Distribution | The age range is wide and not evenly distributed | Create age categories (e.g., 0-18, 19-35, 36-50, etc.) to simplify analysis|
| Gender Representation | Some entries for gender are non-binary or unknown | Handle these cases by either grouping non-binary responses into a category or excluding them |
| Mutiple entries | Ensure there are no issues like multiple hashed emails for the same user. This could indicate duplicates or incorrectly handled data during hashing |  Verify that each hashedEmail corresponds to a unique user |
| Inconsistent Session Lengths | The played_hours column can have values that seem unusually short, suggesting that some players may have logged incomplete sessions | Set thresholds for what constitutes a valid session duration. Filter out sessions that fall below the threshold |
| Correlation Between Variables | Columns like experience and played_hours might have a strong relationship | Use correlation metrics or scatter plots to visualize relationships |

In [8]:
players <- read_csv("./players.csv")

players_recode <- players |>
  mutate(subscribe = as_factor(subscribe)) |>
  mutate(subscribe = fct_recode(subscribe, "Subscribed" = "TRUE", "Not subscribed" = "FALSE"))
players_recode

players_mean <- players |>
select(played_hours, Age) |>
map_df(mean, na.rm = TRUE)



Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<fct>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,Subscribed,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,Subscribed,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,Not subscribed,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,Not subscribed,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,17
Amateur,Not subscribed,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,Subscribed,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


In [ ]:
options(repr.plot.width=12, repr.plot.height=7)
players_plot <- players_recode |> 
ggplot(aes(x = played_hours, y = Age, color = subscribe)) + 
geom_point() +
labs(x = "Time spent playing (hours)", y = "Age of player (years)", title = "Time played and Age (unstandardized)") + 
theme(text = element_text(size = 15))
players_plot

Warning message:
“Removed 2 rows containing missing values or values outside the scale range
(`geom_point()`).”


In [ ]:
players_histogram <- ggplot(players_recode, aes(x = played_hours)) + 
geom_histogram(binwidth = 0.5) +
labs(x = "Time spent playing (hours)", y = "Count", title = "Distribution of playing time") + 
theme(text = element_text(size = 15))
       
players_histogram + scale_x_continuous(limits = c(0, 50)) + scale_y_continuous(limits = c(0, 25))

In [2]:
#Classification algorithm: